# Датаклассы

https://habr.com/ru/post/415829/

Часто в конструкторе класса можно наблюдать следующую ситуацию:

In [1]:
class SomeThing:
    def __init__(self, value1, value2):
        self.value1 = value1
        self.value2 = value2

В Python 3.7 появились классы данных, в которых не нужно писать все эти однотипные присвоения. Они задаются декоратором `@dataclass`

In [2]:
from dataclasses import dataclass

@dataclass
class SomeThing:
    value1: int
    value2: str
        

s = SomeThing(1, "abc")
s.value1, s.value2

(1, 'abc')

In [3]:
SomeThing.value1

AttributeError: type object 'SomeThing' has no attribute 'value1'

Аннотации типов обязательны, иначе поля игнорируются декоратором

In [4]:
value3 = 5

@dataclass
class SomeThing:
    value1: int
    value2: str
    value3
        

s = SomeThing(1, "abc")
s.value1, s.value2, s.value3

AttributeError: 'SomeThing' object has no attribute 'value3'

### make_dataclass

Библиотека dataclass предоставляет функцию, которая позволяет создавать класс данных следующим образом:

In [5]:
from dataclasses import make_dataclass

SomeThing = make_dataclass("SomeThing", ["value1", "value2"])
s = SomeThing(1, "abc")
s.value1, s.value2

(1, 'abc')

### Значения по умолчанию

В синтаксисе датакласса можно указать значения по умолчанию:

In [6]:
@dataclass
class SomeThing:
    value1: int = 1
    value2: str = "abc"
        
a = SomeThing(45)
b = SomeThing(value2="abcde")

print(a.value1, a.value2)
print(b.value1, b.value2)

45 abc
1 abcde


In [7]:
SomeThing.value1

1

Но надо помнить о том, что аргументы по умолчанию всегда должны следовать после всех позиционных аргументов. Это касается и конструктора, генерируемого декоратором dataclass.

In [8]:
@dataclass
class SomeThing:
    value1: int = 1
    value2: str
        
a = SomeThing(45)
b = SomeThing(value2="abcde")

print(a.value1, a.value2)
print(b.value1, b.value2)

TypeError: non-default argument 'value2' follows default argument

### Frozen Data Class

Отличный способ, чтобы хранить константы: объекты датаклассов, созданных таким образом, неизменяемы.

In [9]:
@dataclass(frozen=True)
class SomeThing:
    value1: int
    value2: str
        
        
s = SomeThing(value1=256, value2="abc")
s.value2 = "abcde"

FrozenInstanceError: cannot assign to field 'value2'

In [10]:
s.value3 = 2

FrozenInstanceError: cannot assign to field 'value3'

### Параметры класса данных

- `init: bool = True` - создать или не создать конструктор
- `repr: bool = True` - создать или не создать `__repr__`
- `eq: bool = True` - создать или не создать метод `__eq__`
- `order: bool = False` - создать или не создать методы сравнения объектов
- `unsafe_hash: bool: False` - создать или не создать метод `__hash__`. Само создание метода зависит также от параметров `eq` и `frozen`
- `frozen: bool = False` - запрет изменения атрибутов класса

Посмотрим, что из себя представляют `__eq__`, `__lt__` и остальные, сгенерированные автоматически

In [11]:
@dataclass(eq=True, order=True, unsafe_hash=True)
class SomeThing:
    value1: int
    value2: str

SomeThing(1, 2) == SomeThing(1, 2)

True

In [12]:
SomeThing(1, 2) > SomeThing(2, 1)

False

In [13]:
SomeThing(1, 2) < SomeThing(2, 1)

True

По умолчанию у таких классов сравниваются кортежи значений в том порядке, в котором они были изначально заданы. Посмотреть OrderedDict параметров можно через поле `__dataclass_fields__`

In [14]:
SomeThing(100, "abc").__dataclass_fields__

{'value1': Field(name='value1',type=<class 'int'>,default=<dataclasses._MISSING_TYPE object at 0x7fd4080645e0>,default_factory=<dataclasses._MISSING_TYPE object at 0x7fd4080645e0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=_FIELD),
 'value2': Field(name='value2',type=<class 'str'>,default=<dataclasses._MISSING_TYPE object at 0x7fd4080645e0>,default_factory=<dataclasses._MISSING_TYPE object at 0x7fd4080645e0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=_FIELD)}

А что с хешами?

In [15]:
@dataclass(eq=True, unsafe_hash=True)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

True
False


In [16]:
@dataclass(eq=False, unsafe_hash=True)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

True
False


In [17]:
@dataclass(frozen=True, eq=True, unsafe_hash=True)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

True
False


In [18]:
@dataclass(frozen=True, eq=False, unsafe_hash=True)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

True
False


In [19]:
@dataclass(frozen=True, eq=False)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

True
True


In [20]:
@dataclass(frozen=False, eq=False, unsafe_hash=False)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

True
True


In [21]:
@dataclass(frozen=False, eq=True, unsafe_hash=False)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

TypeError: unhashable type: 'SomeThing'

In [22]:
@dataclass(frozen=True, eq=True, unsafe_hash=False)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

True
False


### Изменяемые значения по умолчанию

Как мы знаем, использовать изменяемые объекты в качестве значений по умолчанию - плохая идея, поскольку они инстанцируются только один раз при объявлении функции. Dataclass это учитывает:

In [23]:
from typing import List

@dataclass
class SomeThing:
    value: List[int] = []

ValueError: mutable default <class 'list'> for field value is not allowed: use default_factory

Библиотека предлагает использовать default_factory:

In [27]:
from dataclasses import field

@dataclass
class SomeThing:
    value: List[int] = field(default_factory=list)
        
SomeThing().value

[]

#### Параметры field
- `default`: значение по умолчанию. Этот параметр необходим, так как вызов `field` заменяет задание значения поля по умолчанию
- `init`: включает (задан по умолчанию) использование поля в методе `__init__`
- `repr`: включает (задан по умолчанию) использование поля в методе `__repr__`
- `compare`: включает (задан по умолчанию) использование поля в методах сравнения (`__eq__`, `__le__` и других)
- `hash`: может быть булевое значение или None. Если он равен True, поле используется при вычислении хэша. Если указано None (по умолчанию) — используется значение параметра compare.
Одной из причин указать `hash=False` при заданном `compare=True` может быть сложность вычисления хэша поля при том, что оно необходимо для сравнения.
- `metadata`: произвольный словарь или None. Значение оборачивается в MappingProxyType, чтобы оно стало неизменяемым. Этот параметр не используется самими классами данных и предназначен для работы сторонних расширений.


### Обработка после инициализации

В классах данных автоматически создается метод `__init__`, в котором исплоняется код присвоения значений в поля объекта: `self.value = value`. Но что если мы хотим использовать датакласс, но дополнить конструктор какими-то еще действиями? Для этого можем задать метод `__post_init__`

In [28]:
@dataclass
class Book:
    title: str
    author: str
    desc: str = None

    def __post_init__(self):
        self.desc = self.desc or "`%s` by %s" % (self.title, self.author)
        
        
Book("Название", "Автор").desc

'`Название` by Автор'

В этом методе можно использовать дополнительные аргументы конструктора, которые не нужно записывать в self. Для этого предназначен класс `dataclasses.InitVar`:

In [34]:
from dataclasses import InitVar

@dataclass
class Book:
    title: str
    author: str
    gen_desc: InitVar[bool] = True
    desc: str = None

    def __post_init__(self, gen_desc: bool):
        if gen_desc and self.desc is None:
            self.desc = "`%s` by %s" % (self.title, self.author)
            
            
print(Book("Название", "Автор", True).desc)
print(Book("Название", "Автор", False).desc)

`Название` by Автор
None


In [37]:
Book("a", "b", False).gen_desc

True

### Наследование в датаклассах

Если датакласс наследуется от другого датакласса, то он складывает OrderedDict'ы обоих классов и использует полученный общий OrderedDict во всех генерируемых методах.

In [38]:
from typing import Any


@dataclass
class BaseBook:
    title: Any = None
    author: str = None

@dataclass
class Book(BaseBook):
    desc: str = None
    title: str = "Unknown"
        
        
Book()

Book(title='Unknown', author=None, desc=None)

In [39]:
Book().__dataclass_fields__

{'title': Field(name='title',type=<class 'str'>,default='Unknown',default_factory=<dataclasses._MISSING_TYPE object at 0x7fd4080645e0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=_FIELD),
 'author': Field(name='author',type=<class 'str'>,default=None,default_factory=<dataclasses._MISSING_TYPE object at 0x7fd4080645e0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=_FIELD),
 'desc': Field(name='desc',type=<class 'str'>,default=None,default_factory=<dataclasses._MISSING_TYPE object at 0x7fd4080645e0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=_FIELD)}